In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
import glob
import re
import json
import os.path
from urllib.parse import urlparse


In [2]:
inlinks_parth = {}

In [3]:
with open("inlinks_parth.json") as f:
    inlinks_parth = json.load(f)

In [4]:
inlinks_priya = {}

In [5]:
with open("inlinks_priya.json") as f:
    inlinks_priya = json.lohwad(f)

In [6]:
mapping = {
    "settings" : {
        "index": {
          "store": {
            "type": "fs"
          },
          "number_of_shards": 2,
          "number_of_replicas": 1
        },
        "analysis": {
          "analyzer": {
            "my_english": { 
              "type": "english",
              "stopwords_path": "stoplist.txt" 
            }
          }
        }
    },
    "mappings": {
        "doc": { 
          "properties": { 
            "DOCNO": { "type" : "text"  }, 
            "TITLE": { "type" : "text"},
            "TEXT": { 
                "type": "text",
                "store": True,
                "fielddata": True,
                "term_vector": "with_positions_offsets_payloads",
                "analyzer": "my_english"
            }, 
            "HTML": { "type": "text"  },
            "INLINK" : { "type": "text"  },
            "OUTLINK": { "type": "text"  },
            "AUTHOR": {"type" : "text"},
#             "DEPTH" : {"type":"integer"}

          }
        }
    }
}

In [7]:
es = Elasticsearch()
es.indices.create('crawled_data_merged', body=mapping)

In [8]:
id_scheme_removed = []

In [9]:
pro_path = "proc/"

In [10]:
c = 0
for i in range(1, 20001):
    file_path = pro_path + str(i) + ".json"
    with open(file_path, "r") as f:
        data = json.load(f)
        
        parsedUrl = urlparse(data["DOCNO"].lower())
    
        json_d = {}

#         if(parsedUrl.netloc + parsedUrl.path) in id_scheme_removed:
#             json_d["TEXT"] = " "
#             print(parsedUrl.netloc + parsedUrl.path)
#         if(parsedUrl.netloc + parsedUrl.path) not in id_scheme_removed:
#             id_scheme_removed.append(parsedUrl.netloc + parsedUrl.path)
#             json_d["TEXT"] = data["TEXT"]
        

        json_d["DOCNO"] = data["DOCNO"]
        json_d["TEXT"] = data["TEXT"]
#         json_d["HTML"] = str(data["HTML"])
        json_d["HTML"] = ""
        json_d["INLINK"] = []
        
        
        
        # 
    
        if data['DOCNO'] in inlinks_parth:
            json_d["INLINK"] = json_d["INLINK"] + inlinks_parth[data['DOCNO']]
            
        if data['DOCNO'] in inlinks_priya:
            json_d["INLINK"] = json_d["INLINK"] + inlinks_priya[data['DOCNO']]
            
        json_d["INLINK"] = list(set(json_d["INLINK"]))

        json_d["OUTLINK"] = list(set(data["OUTLINK"]))
        json_d["AUTHOR"] = ["Parth"]
        json_d["TITLE"] = data["TITLE"]
        
        res = es.get(index = "crawled_data_merged", 
                     doc_type = "doc",
                     id = json_d['DOCNO'],
                     ignore = 404)
                                                                     
        if res["found"] == True:
#             if(res["_source"]["TEXT"] == " "):
#                 json_d["TEXT"] = " "
            
            json_d["INLINK"] = list(set( res["_source"]["INLINK"] + json_d["INLINK"]))
            json_d["OUTLINK"] = list(set( res["_source"]["OUTLINK"] + json_d["OUTLINK"]))
            json_d["AUTHOR"] =  list(set( res["_source"]["AUTHOR"] + json_d["AUTHOR"]))
        
            
        res = es.index(index="crawled_data_merged", doc_type='doc', id=json_d['DOCNO'], body=json_d)
        


In [11]:
# res = es.get(index="crawled_data_test", doc_type='doc', id=123, ignore=404)

In [12]:
# res

In [13]:
# res['_source']['INLINK']

In [14]:
# es = Elasticsearch()
# es.indices.create('crawled_data_merged', body=mapping)

In [15]:
# inlinks_parth = []

In [16]:
# es = Elasticsearch()
# es.indices.create('crawled_data_merged', body=mapping)